## We check if there has been improvement in classification results with the amount we've converted 
We've summarized about half of the data now, but it's enough to make a start on classification.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import re
import numpy as np
import pandas as pd

In [75]:
file = '../data/product_merged.csv'
df = pd.read_csv(file, usecols=['Complaint ID', 'Product', 'Issue', 'Consumer complaint narrative'])

In [90]:
text_file = '../output/summary_1.txt'
text = []
with open(text_file, 'r') as f:
    for line in f:
        text.append(line)
text_file = '../output/summary_100000.txt'
with open(text_file, 'r') as f:
    for line in f:
        text.append(line)
text_file = '../output/summary_200000.txt'
with open(text_file, 'r') as f:
    for line in f:
        text.append(line)
text_file = '../output/summary_300000.txt'
with open(text_file, 'r') as f:
    for line in f:
        text.append(line)

In [98]:
no_newlines = [re.sub(r'\n', '', doc) for doc in text]
summary_df = pd.DataFrame()
summary_df['summary'] = no_newlines
summary_df.to_csv('../output/all_summaries.csv')
df['summarized'] = no_newlines
df.to_csv('../output/with_summarized.csv', index=False)

## Machine Learning Models

In [97]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [105]:
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_sm")

In [103]:
import re
special = re.compile(r'http\S+|www\S+|[^a-zA-Z ]+|xx+')
docs_orig = [' '.join(special.sub('', doc.lower()).split()) for doc in df['Consumer complaint narrative'].values]
docs_summ = [' '.join(special.sub('', doc.lower()).split()) for doc in df['summarized'].values]

In [ ]:
tokenized_orig = []
for doc in nlp.pipe(docs_orig, disable=['tagger', 'parser', 'ner']):
    tokenized_orig.append(" ".join(token.lemma_.lower() for token in doc if not token.is_stop and not token.is_space \
            and not token.is_punct and not token.like_num))
    
tokenized_summ = []
for doc in nlp.pipe(docs_summ, disable=['tagger', 'parser', 'ner']):
    tokenized_summ.append(" ".join(token.lemma_.lower() for token in doc if not token.is_stop and not token.is_space \
            and not token.is_punct and not token.like_num))

In [35]:
del docs_orig
del docs_summ

In [36]:
df['orig'] = tokenized_orig
df['summ'] = tokenized_summ
df = valid_df.dropna()
df.to_csv('../output/with_summarized_tokenized.csv', index=False)

/Users/sueliu/Mudano/customer-complaints/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/sueliu/Mudano/customer-complaints/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Now some ML

In [56]:
valid_df = valid_df[valid_df.Product.isin(valid_targets)]

In [57]:
train_df, test_df = train_test_split(valid_df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [58]:
X_train_orig, X_val_orig = train_df['orig'].values, val_df['orig'].values
X_train_summ, X_val_summ = train_df['summ'].values, val_df['summ'].values
y_train, y_val = train_df['Product'].values, val_df['Product'].values

In [75]:
X_train_orig[0]

'fall financial hardship lose job file bankruptcy chapter discharge time child afford payment trustee month mortgage miss approx mortgage payment foreclosure proceed begin occur anytime protect automatic stay bankruptcy law well fargo mortgage approve remodification new federal law fha loan charge exorbitantly high capitalize interest note new money page honest naively impression principal balance remain file chapter child keep house important accept remodification sign document local well fargo branch know hit hard delinquent interest unknown fee feel well gouge place underwater loan feel offer well finance rate new money cap low annual rate potential difference month big deal family thank advance help provide matt sincerely'

In [76]:
X_train_summ[0]

'bank america report equity line derogatory item way report credit bureau late adversely affect credit score unable refinance property value property drop credit report fault drop credit report'

In [59]:
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=1000)
tfidf_vect.fit(X_train_orig)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=1000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [60]:
X_train_orig_tfidf = tfidf_vect.transform(X_train_orig)
X_val_orig_tfidf = tfidf_vect.transform(X_val_orig)

Now for the summarised

In [69]:
tfidf_vect2 = TfidfVectorizer(analyzer='word', max_features=1000)
tfidf_vect2.fit(X_train_summ)
X_train_summ_tfidf = tfidf_vect2.transform(X_train_summ)
X_val_summ_tfidf = tfidf_vect2.transform(X_val_summ)

### Logistic Regression

In [48]:
from sklearn.linear_model import LogisticRegression

In [62]:
lr = LogisticRegression(C=1.0, max_iter=500, class_weight='balanced', multi_class='auto', solver='lbfgs', n_jobs=3)
lr.fit(X_train_orig_tfidf, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=500, multi_class='auto', n_jobs=3, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [63]:
valid_targets = ['Credit card or prepaid card', 'Debt collection', 'Mortgage', 
                 'Payday loan, title loan, or personal loan', 'Student loan']
targets = sorted(val_df['Product'].unique())

In [64]:
pred_orig = lr.predict(X_val_orig_tfidf)

In [65]:
print(classification_report(y_val, pred_orig))

                                           precision    recall  f1-score   support

              Credit card or prepaid card       0.58      0.91      0.71      1419
                          Debt collection       0.97      0.85      0.91     15500
                                 Mortgage       0.96      0.93      0.94      9019
Payday loan, title loan, or personal loan       0.38      0.80      0.52      1077
                             Student loan       0.89      0.91      0.90      3692

                                 accuracy                           0.88     30707
                                macro avg       0.75      0.88      0.79     30707
                             weighted avg       0.92      0.88      0.89     30707



In [70]:
lr_summ = LogisticRegression(C=1.0, max_iter=500, class_weight='balanced', multi_class='auto', solver='lbfgs', n_jobs=3)
lr_summ.fit(X_train_summ_tfidf, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=500, multi_class='auto', n_jobs=3, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [71]:
pred_summ = lr.predict(X_val_summ_tfidf)

In [72]:
print(classification_report(y_val, pred_summ))

                                           precision    recall  f1-score   support

              Credit card or prepaid card       0.20      0.32      0.25      1419
                          Debt collection       0.54      0.43      0.48     15500
                                 Mortgage       0.31      0.35      0.33      9019
Payday loan, title loan, or personal loan       0.11      0.22      0.14      1077
                             Student loan       0.14      0.13      0.14      3692

                                 accuracy                           0.36     30707
                                macro avg       0.26      0.29      0.27     30707
                             weighted avg       0.39      0.36      0.37     30707

